### Accessing workspace

In [ ]:
from azureml.core import Workspace

ws = Workspace.get(name="your-workspace-name",
                subscription_id='your-subscription-id',
                resource_group='your-resource-group')
ws

### Creating AKS cluster using default configuration
you can customize by providing parameters to [provisioning_configuration()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.akscompute?view=azure-ml-py#attach-configuration-resource-group-none--cluster-name-none--resource-id-none--cluster-purpose-none-) as well 

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

prov_config = AksCompute.provisioning_configuration(vm_size = "STANDARD_DS3_V2",
                                                       agent_count = 1,
                                                       location = "westus2",
                                                    cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)

aks_name = 'myaks'
# Creates the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Waits for the create process to complete
aks_target.wait_for_completion(show_output = True)


### Attaching existing cluster

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

resource_group = 'your-resource-group'
cluster_name = 'your-aks'
                                       )
attach_config = AksCompute.attach_configuration(resource_group = resource_group,
                                         cluster_name = cluster_name,
                                               cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
aks_target = ComputeTarget.attach(ws, cluster_name, attach_config)

In [ ]:
aks_target

### Deploy to AKS

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

yourenv = Environment(name="your-env")

dependencies = CondaDependencies()
dependencies.set_python_version("3.6.6")
dependencies.add_pip_package("scikit-learn==0.20.3")
dependencies.add_pip_package("numpy==1.16.2")
dependencies.add_pip_package("pandas==0.23.4")
dependencies.add_pip_package("joblib==0.14.1")

myenv.python.conda_dependencies=dependencies

inference_config = InferenceConfig(entry_script='score.py', environment=yourenv)


In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute

aks_target = AksCompute(ws,cluster_name)
aks_target

In [ ]:
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
model = Model(ws, name='regression_model')
service = Model.deploy(ws, "your-service", [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)


In [ ]:
print(service.get_logs())
